In [1]:
import gzip
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import urllib.request

# Função para ler o arquivo do MNIST
def read_mnist(images_path, labels_path):
    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(labels), 784)

    return images, labels

# Baixar e ler o conjunto de dados MNIST
url_base = 'http://yann.lecun.com/exdb/mnist/'
train_images_url = url_base + 'train-images-idx3-ubyte.gz'
train_labels_url = url_base + 'train-labels-idx1-ubyte.gz'
test_images_url = url_base + 't10k-images-idx3-ubyte.gz'
test_labels_url = url_base + 't10k-labels-idx1-ubyte.gz'

urllib.request.urlretrieve(train_images_url, 'train-images.gz')
urllib.request.urlretrieve(train_labels_url, 'train-labels.gz')
urllib.request.urlretrieve(test_images_url, 'test-images.gz')
urllib.request.urlretrieve(test_labels_url, 'test-labels.gz')

train_images, train_labels = read_mnist('train-images.gz', 'train-labels.gz')
test_images, test_labels = read_mnist('test-images.gz', 'test-labels.gz')

# Pré-processar os dados
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Definir o modelo
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
model.fit(train_images, train_labels, epochs=5, batch_size=64)

# Avaliar o modelo
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Acurácia do teste:', test_acc)


Epoch 1/5
938/938 [==============================] - 66s 68ms/step - loss: 0.1831 - accuracy: 0.9451
Epoch 2/5
938/938 [==============================] - 57s 61ms/step - loss: 0.0505 - accuracy: 0.9846
Epoch 3/5
938/938 [==============================] - 57s 61ms/step - loss: 0.0363 - accuracy: 0.9882
Epoch 4/5
938/938 [==============================] - 58s 62ms/step - loss: 0.0286 - accuracy: 0.9908
Epoch 5/5
313/313 [==============================] - 4s 11ms/step - loss: 0.0347 - accuracy: 0.9882
Acurácia do teste: 0.9882000088691711
